# 모델저장 및 복원 방법

## 1. Callback 기능을 이용한 Check Point 모델저장 및 복원

In [1]:
### 학습 조기 종료 Early Stopping
import os

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

print(tf.version.VERSION)

2.6.0


In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000]/255.0
test_images = test_images[:1000]/255.0

train_images = train_images[:, :, :, np.newaxis]
test_images = test_images[:, :, :, np.newaxis]

In [3]:
# CNN 인공신경망의 정의

def create_model():
    model = keras.models.Sequential( [
        keras.layers.Conv2D(input_shape = (28, 28, 1),
                        kernel_size = (3,3), padding = 'same', 
                        filters = 32),
        keras.layers.MaxPooling2D((2, 2), strides=2),
        keras.layers.Conv2D(kernel_size = (3,3), padding ='same', 
                        filters = 64),
        keras.layers.MaxPooling2D((2, 2), strides=2),
        keras.layers.Conv2D(kernel_size = (3,3), padding = 'same', 
                        filters = 32),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation = 'relu'),
        keras.layers.Dense(32, activation = 'relu'),
        keras.layers.Dense(10, activation = 'softmax'),
    ])
    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    return model

In [4]:
# 모델 객체를 만듭니다
model = create_model()

# 모델 구조를 출력합니다
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          18464     
_________________________________________________________________
flatten (Flatten)            (None, 1568)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               2

### 모델의 가중치를 저장하는 callback 만들기

In [5]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
cp = ModelCheckpoint(filepath=checkpoint_path,
                              save_weights_only=True,
                              verbose=1)

# 새로운 콜백으로 모델 훈련하기
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp])  # 콜백을 훈련에 전달합니다

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

Epoch 1/10
32/32 [==============================] - 2s 32ms/step - loss: 1.5484 - accuracy: 0.4250 - val_loss: 0.9772 - val_accuracy: 0.6500

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 1s 27ms/step - loss: 0.8339 - accuracy: 0.7020 - val_loss: 0.7824 - val_accuracy: 0.7130

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 1s 27ms/step - loss: 0.6417 - accuracy: 0.7700 - val_loss: 0.7600 - val_accuracy: 0.7180

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 1s 27ms/step - loss: 0.5619 - accuracy: 0.8060 - val_loss: 0.6302 - val_accuracy: 0.7700

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 1s 26ms/step - loss: 0.4757 - accuracy: 0.8380 - val_loss: 0.6147 - val_accuracy: 0.7880

Epoch 00005: saving model to training_1/cp.ckpt
Epoch 6/10
32/32 [==============================] 

In [6]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'cp.ckpt.index', 'checkpoint']

![checkpoint1](img/checkpoint1.jpg)

### 이제 훈련되지 않은 새로운 모델을 다시 빌드하고 테스트 세트에서 평가합니다. 훈련되지 않은 모델은 확률 수준(~10% 정확도)에서 수행됩니다.

In [7]:
# 기본 모델 객체를 만듭니다
model = create_model()

# 모델을 평가합니다
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.2921 - accuracy: 0.1500
훈련되지 않은 모델의 정확도: 15.00%


### 두 모델이 동일한 아키텍처를 공유하기만 한다면 두 모델 간에 가중치를 공유할 수 있습니다. 따라서 가중치 전용에서 모델을 복원할 때 원래 모델과 동일한 아키텍처로 모델을 만든 다음 가중치를 설정합니다.

In [8]:
# 가중치 로드
model.load_weights(checkpoint_path)

# 모델 재평가
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.5840 - accuracy: 0.8000
복원된 모델의 정확도: 80.00%


In [9]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}-{val_loss:.2f}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 다섯 번째 에포크마다 가중치를 저장하기 위한 콜백을 만듭니다
cp = ModelCheckpoint(filepath=checkpoint_path, 
                                verbose=1, 
                                save_weights_only=True,
                                period=2)

# 새로운 모델 객체를 만듭니다
model = create_model()

# `checkpoint_path` 포맷을 사용하는 가중치를 저장합니다
model.save_weights(checkpoint_path.format(epoch=0,val_loss=0.0))

# 새로운 콜백을 사용하여 모델을 훈련합니다
model.fit(train_images, 
          train_labels,
          epochs=10, 
          validation_data=(test_images,test_labels),
          callbacks=[cp],
          verbose=1)

Epoch 1/10
32/32 [==============================] - 1s 31ms/step - loss: 1.5542 - accuracy: 0.4120 - val_loss: 0.9174 - val_accuracy: 0.6910
Epoch 2/10
32/32 [==============================] - 1s 27ms/step - loss: 0.8711 - accuracy: 0.6960 - val_loss: 0.8598 - val_accuracy: 0.6850

Epoch 00002: saving model to training_2/cp-0002-0.86.ckpt
Epoch 3/10
32/32 [==============================] - 1s 27ms/step - loss: 0.7035 - accuracy: 0.7460 - val_loss: 0.6899 - val_accuracy: 0.7530
Epoch 4/10
32/32 [==============================] - 1s 27ms/step - loss: 0.6021 - accuracy: 0.7740 - val_loss: 0.6462 - val_accuracy: 0.7670

Epoch 00004: saving model to training_2/cp-0004-0.65.ckpt
Epoch 5/10
32/32 [==============================] - 1s 27ms/step - loss: 0.4986 - accuracy: 0.8130 - val_loss: 0.6540 - val_accuracy: 0.7760
Epoch 6/10
32/32 [==============================] - 1s 27ms/step - loss: 0.4177 - accuracy: 0.8560 - val_loss: 0.6834 - val_accuracy: 0.7700

Epoch 00006: saving model to traini

In [10]:
os.listdir(checkpoint_dir)

['cp-0000-0.00.ckpt.data-00000-of-00001',
 'cp-0000-0.00.ckpt.index',
 'cp-0010-0.64.ckpt.index',
 'cp-0002-0.86.ckpt.data-00000-of-00001',
 'cp-0004-0.65.ckpt.data-00000-of-00001',
 'cp-0008-0.58.ckpt.index',
 'cp-0006-0.68.ckpt.data-00000-of-00001',
 'cp-0008-0.58.ckpt.data-00000-of-00001',
 'cp-0002-0.86.ckpt.index',
 'cp-0010-0.64.ckpt.data-00000-of-00001',
 'cp-0006-0.68.ckpt.index',
 'cp-0004-0.65.ckpt.index',
 'checkpoint']

![checkpoint2](img/checkpoint2.jpg)

In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0010-0.64.ckpt'

In [12]:
# 새로운 모델 객체를 만듭니다
model = create_model()

# 이전에 저장한 가중치를 로드합니다
model.load_weights(latest)

# 모델을 재평가합니다
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.6416 - accuracy: 0.8000
복원된 모델의 정확도: 80.00%


## 2. Callback 기능과 Early Stopping을 활용한 Check Point 모델 저장

In [13]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_3/cp-{epoch:04d}-{val_loss:.2f}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 다섯 번째 에포크마다 가중치를 저장하기 위한 콜백을 만듭니다
early_stopping = EarlyStopping(monitor='val_loss',patience=5)
cp = ModelCheckpoint(filepath=checkpoint_path,
                                save_weights_only=True,
                                verbose=1,
                                save_best_only=True)

# 새로운 모델 객체를 만듭니다
model = create_model()

# 새로운 콜백을 사용하여 모델을 훈련합니다
model.fit(train_images, 
          train_labels,
          epochs=50, 
          validation_data=(test_images,test_labels),
          callbacks=[early_stopping, cp],
          verbose=0)


Epoch 00001: val_loss improved from inf to 1.26733, saving model to training_3/cp-0001-1.27.ckpt

Epoch 00002: val_loss improved from 1.26733 to 0.75793, saving model to training_3/cp-0002-0.76.ckpt

Epoch 00003: val_loss did not improve from 0.75793

Epoch 00004: val_loss improved from 0.75793 to 0.66036, saving model to training_3/cp-0004-0.66.ckpt

Epoch 00005: val_loss did not improve from 0.66036

Epoch 00006: val_loss improved from 0.66036 to 0.64664, saving model to training_3/cp-0006-0.65.ckpt

Epoch 00007: val_loss improved from 0.64664 to 0.56045, saving model to training_3/cp-0007-0.56.ckpt

Epoch 00008: val_loss did not improve from 0.56045

Epoch 00009: val_loss did not improve from 0.56045

Epoch 00010: val_loss did not improve from 0.56045

Epoch 00011: val_loss did not improve from 0.56045

Epoch 00012: val_loss did not improve from 0.56045


In [14]:
os.listdir(checkpoint_dir)

['cp-0002-0.76.ckpt.data-00000-of-00001',
 'cp-0006-0.65.ckpt.index',
 'cp-0001-1.27.ckpt.index',
 'cp-0006-0.65.ckpt.data-00000-of-00001',
 'cp-0001-1.27.ckpt.data-00000-of-00001',
 'cp-0004-0.66.ckpt.data-00000-of-00001',
 'cp-0002-0.76.ckpt.index',
 'cp-0004-0.66.ckpt.index',
 'cp-0007-0.56.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0007-0.56.ckpt.index']

![cp_earlystopping](img/cp_earlystopping.jpg)

In [15]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

training_3/cp-0007-0.56.ckpt


In [16]:
# 새로운 모델 객체를 만듭니다
model = create_model()

# 이전에 저장한 가중치를 로드합니다
model.load_weights(latest)

# 모델을 재평가합니다
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.5604 - accuracy: 0.8050
복원된 모델의 정확도: 80.50%


## Check Point 모델 저장 파일
### 위의 코드는 이진 형식의 훈련된 가중치만 포함하는 check point 형식의 파일 모음에 가중치를 저장합니다. check point에는 다음이 포함됩니다.

### (1) 모델의 가중치를 포함하는 하나 이상의 샤드(checkpoint)
### (2) 어떤 가중치가 어떤 샤드에 저장되어 있는지 나타내는 인덱스 파일(~.ckpt.index)
### (3) 단일 머신에서 모델을 훈련하는 경우 접미사가 .data-00000-of-00001인 하나의 샤드를 갖게 됩니다.

In [17]:
# 가중치를 저장합니다
model.save_weights('checkpoints/my_checkpoint')

# 새로운 모델 객체를 만듭니다
model = create_model()

# 가중치를 복원합니다
model.load_weights('checkpoints/my_checkpoint')

# 모델을 평가합니다
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.5604 - accuracy: 0.8050
복원된 모델의 정확도: 80.50%


## 3. SavedModel을 이용한 전체 모델 저장(모델의 구조, 가중치)

In [18]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# SavedModel로 전체 모델을 저장합니다
#!mkdir -p saved_model
model.save('saved_model/my_model/')

Epoch 1/5
32/32 [==============================] - 1s 23ms/step - loss: 1.4458 - accuracy: 0.4640
Epoch 2/5
32/32 [==============================] - 1s 23ms/step - loss: 0.8064 - accuracy: 0.7110
Epoch 3/5
32/32 [==============================] - 1s 23ms/step - loss: 0.6986 - accuracy: 0.7470
Epoch 4/5
32/32 [==============================] - 1s 22ms/step - loss: 0.5634 - accuracy: 0.8090
Epoch 5/5
32/32 [==============================] - 1s 23ms/step - loss: 0.4788 - accuracy: 0.8360
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [19]:
# my_model 디렉토리
os.listdir('saved_model')
# assests 폴더, saved_model.pb, variables 폴더
os.listdir('saved_model/my_model/')

['variables', 'assets', 'keras_metadata.pb', 'saved_model.pb']

![savedmodel](img/savedmodel.jpg)

In [20]:
new_model = tf.keras.models.load_model('saved_model/my_model/')

# 복원된 모델을 재훈련합니다.
new_model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
32/32 [==============================] - 1s 21ms/step - loss: 0.4923 - accuracy: 0.8290
Epoch 2/5
32/32 [==============================] - 1s 21ms/step - loss: 0.3608 - accuracy: 0.8770
Epoch 3/5
32/32 [==============================] - 1s 21ms/step - loss: 0.3225 - accuracy: 0.8870
Epoch 4/5
32/32 [==============================] - 1s 21ms/step - loss: 0.2741 - accuracy: 0.9020
Epoch 5/5
32/32 [==============================] - 1s 21ms/step - loss: 0.2350 - accuracy: 0.9210


In [21]:
# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.5763 - accuracy: 0.8150
복원된 모델의 정확도: 81.50%
(1000, 10)


## 4. HDF5 Model을 이용한 전체 모델 저장(모델의 구조, 가중치)

In [22]:
# hdf5 폴더에 모델을 저장합니다
# !mkdir -p hdf5_model

# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
# '.h5' 확장자는 이 모델이 HDF5로 저장되었다는 것을 나타냅니다
model.save('hdf5/my_model.h5')

Epoch 1/5
32/32 [==============================] - 1s 21ms/step - loss: 1.3474 - accuracy: 0.5340
Epoch 2/5
32/32 [==============================] - 1s 21ms/step - loss: 0.7341 - accuracy: 0.7540
Epoch 3/5
32/32 [==============================] - 1s 21ms/step - loss: 0.5619 - accuracy: 0.8000
Epoch 4/5
32/32 [==============================] - 1s 21ms/step - loss: 0.4961 - accuracy: 0.8260
Epoch 5/5
32/32 [==============================] - 1s 22ms/step - loss: 0.3989 - accuracy: 0.8550


In [23]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = tf.keras.models.load_model('hdf5/my_model.h5')

# 모델 구조를 출력합니다
new_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 7, 7, 32)          18464     
_________________________________________________________________
flatten_8 (Flatten)          (None, 1568)              0         
_________________________________________________________________
dense_24 (Dense)             (None, 128)              

In [24]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.5697 - accuracy: 0.8020
복원된 모델의 정확도: 80.20%
